In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

In [ ]:
! pip install wandb
! pip install datasets
! pip install transformers


In [ ]:
!pip install nvidia-smi
!nvidia-smi

In [4]:
%cd /content/drive/MyDrive/goorm/goorm-project-2-korean_mrc/Best-Fit code

/content/drive/MyDrive/goorm/goorm-project-2-korean_mrc/Best-Fit code


In [ ]:
!ls

Best-Fit_code.ipynb  inference_KoB.py  out		 train_KoB.py
data		     max_answer_40     __pycache__	 wandb
dataset.py	     modules	       requirements.txt


In [11]:
!python ./modules/preprocess_KoB.py --data_path './data' --save_path './data' --pretrained_model_name 'monologg/kobigbird-bert-base' --max_length 1024 --train_fn 'pre_train.json'

Train shape (10434, 5)
train length: 8347, validation length: 2087
finished preprocessing


In [12]:
import torch

torch.cuda.empty_cache()

In [ ]:
!python ./train_KoB.py --model_fn 'pre_train_Data' --file_path './data' --pretrained_model_name 'monologg/kobigbird-bert-base' --n_epochs 2 --batch_size_per_device 4 --n_best 5 --max_answer_length 15

In [ ]:
!ls -a

Baseline_Train_Data  inference_KoB.py		 __pycache__
Best-Fit_code.ipynb  max_answer_15_pre_news_2ep  requirements.txt
.cache		     max_answer_40		 train_KoB.py
.checkpoints	     max_answer_40_news		 wandb
data		     modules
dataset.py	     out


In [19]:
!python ./inference_KoB.py --model_fn 'pre_train_Data' --file_path './data' --pretrained_model_name 'monologg/kobigbird-bert-base' --n_best 5 --max_answer_length 15 --output_name 'pre_train_dev.csv'

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'bert.pooler.bias']
- This IS expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of 

In [15]:
def levenshtein(s1, s2):
    # s2가 s1보다 길면 반대로 돌려서 계산
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    
    # s2의 길이가 0 이면 모두 추가 
    if len(s2) == 0:
        return len(s1)

    # base 행 : [0,1,2...s2]
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1): # s1에서 글자 하나씩 빼옴
        current_row = [i + 1] # 0~i 길이만큼 deletion이 일어났을 때의 비용
        for j, c2 in enumerate(s2): # s2에서 글자 하나씩 빼옴
            # 비용 계산
            insertions = previous_row[j + 1] + 1 
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        previous_row = current_row

    # s1 과 s2 끝부분끼리 비교부분 리턴
    return previous_row[-1]


In [ ]:
def dev2csv(dataset, long_short):
    with open("dev.csv", 'w') as fd:
        writer = csv.writer(fd)
        writer.writerow(['Id', 'Predicted'])

        rows = [[sample['guid'], sample['answers'][long_short]['text']] for sample in dataset ]
        
        writer.writerows(rows)

In [16]:
def return2distance(data1 = "dev.csv", data2 = "baseline.csv"):
    try:
        df1 = pd.read_csv(data1, encoding = 'utf-8')
        df2 = pd.read_csv(data2, encoding = 'utf-8')
    except FileNotFoundError as e: 
        print(e)

    diff = []

    for s1, s2 in zip(df1['Predicted'], df2['Predicted']):
        if type(s2) == float:
            s2 = str(s2)
        if type(s1) == float:
            s1 = str(s1)
    
        diff.append(levenshtein(s1, s2))

    return sum(diff) / len(diff)


In [17]:
import pandas as pd

In [20]:
return2distance(data1='./out/dev.csv', data2='./out/pre_train_dev.csv') #submission_best_fit.csv, result_votting.csv

2.068998562529947

In [22]:
return2distance(data1='./out/pre_train_.csv', data2='./out/submission_best_fit.csv') #submission_best_fit.csv, result_votting.csv

1.435878243512974